## 1. Importy a příprava

In [ ]:
import numpy as np
import random
import pickle
from pathlib import Path
from multipong.ai import RLPongEnv, encode_state, update_q_value, get_q_value

# Nastavení seed pro reprodukovatelnost
random.seed(42)
np.random.seed(42)

## 2. Inicializace prostředí a hyperparametrů

In [ ]:
# Prostředí
env = RLPongEnv(width=400, height=300, paddle_height=60, ball_speed=4)

# Hyperparametry Q-learningu
episodes = 2000
alpha = 0.15      # Learning rate
gamma = 0.95      # Discount factor
epsilon = 0.2     # Exploration rate
decay = 0.995     # Epsilon decay (pokud chceš snižovat exploraci)
num_bins = 8      # Počet binů pro diskretizaci

# Akce
ACTIONS = [0, 1, 2]  # 0=stay, 1=up, 2=down

# Q-tabulka
Q = {}

print(f"🎮 Trénink RL agenta na RLPongEnv")
print(f"📊 Hyperparametry: α={alpha}, γ={gamma}, ε={epsilon}")
print(f"🔁 Epizod: {episodes}")

## 3. Tréninková smyčka Q-learningu

In [ ]:
rewards_per_episode = []
current_epsilon = epsilon

for ep in range(episodes):
    state = env.reset()
    state_key = encode_state(state, env, num_bins=num_bins)
    total_reward = 0
    steps = 0

    done = False
    while not done and steps < 500:  # max 500 kroků na epizodu
        # Epsilon-greedy: explorace vs. exploatace
        if random.random() < current_epsilon:
            action_idx = random.randint(0, len(ACTIONS) - 1)
        else:
            q_vals = get_q_value(Q, state_key)
            action_idx = int(np.argmax(q_vals))

        action = ACTIONS[action_idx]
        next_state, reward, done = env.step(action)
        total_reward += reward
        steps += 1

        next_state_key = encode_state(next_state, env, num_bins=num_bins)

        # Q-learning update
        update_q_value(Q, state_key, action_idx, next_state_key, reward, alpha=alpha, gamma=gamma)

        state_key = next_state_key

    rewards_per_episode.append(total_reward)
    current_epsilon *= decay  # Snižuj exploraci

    # Progress report
    if (ep + 1) % 200 == 0:
        avg_reward = np.mean(rewards_per_episode[-100:])
        print(f"Ep {ep+1:4d}/{episodes} | Avg reward (100): {avg_reward:7.2f} | ε: {current_epsilon:.4f}")

print(f"✅ Trénink dokončen! Q-tabulka má {len(Q)} stavů.")

## 4. Analýza výsledků

In [ ]:
# Statistika
print(f"\n📈 Statistika tréninku:")
print(f"  Počet stavů v Q-tabulce: {len(Q)}")
print(f"  Počet akcí: {len(ACTIONS)}")
print(f"  Průměrná odměna (prvních 100): {np.mean(rewards_per_episode[:100]):.2f}")
print(f"  Průměrná odměna (posledních 100): {np.mean(rewards_per_episode[-100:]):.2f}")
print(f"  Min. odměna: {np.min(rewards_per_episode):.2f}")
print(f"  Max. odměna: {np.max(rewards_per_episode):.2f}")

## 5. Vizualizace křivky učení

In [ ]:
try:
    import matplotlib.pyplot as plt

    # Vyhlazená křivka (klouzavý průměr)
    window = 50
    smoothed = [
        np.mean(rewards_per_episode[max(0, i - window) : i + 1])
        for i in range(len(rewards_per_episode))
    ]

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Graf 1: Surové a vyhlazené odměny
    axes[0].plot(rewards_per_episode, alpha=0.3, label="Surové odměny")
    axes[0].plot(smoothed, linewidth=2, label="Vyhlazeno (klouzavý průměr)")
    axes[0].set_xlabel("Epizoda")
    axes[0].set_ylabel("Celková odměna za epizodu")
    axes[0].set_title("Krátka učení RL agenta")
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Graf 2: Průměr posledních N epizod
    window2 = 100
    running_avg = [
        np.mean(rewards_per_episode[max(0, i - window2) : i + 1])
        for i in range(len(rewards_per_episode))
    ]
    axes[1].plot(running_avg, linewidth=2, color="green")
    axes[1].set_xlabel("Epizoda")
    axes[1].set_ylabel("Průměrná odměna (100 epizod)")
    axes[1].set_title("Konvergence - průměr posledních 100 epizod")
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()
    print("✅ Grafy vykresleny")

except ImportError:
    print("⚠️ matplotlib není nainstalován, grafy se nevykresluji")

## 6. Evaluace natrénovaného agenta

In [ ]:
# Evaluace bez explorace (pure exploitation)
eval_episodes = 100
eval_rewards = []

for ep in range(eval_episodes):
    state = env.reset()
    state_key = encode_state(state, env, num_bins=num_bins)
    total_reward = 0
    done = False
    steps = 0

    while not done and steps < 500:
        # Vždy zvolíme best action (greedy)
        q_vals = get_q_value(Q, state_key)
        action_idx = int(np.argmax(q_vals))
        action = ACTIONS[action_idx]

        next_state, reward, done = env.step(action)
        total_reward += reward
        steps += 1

        state_key = encode_state(next_state, env, num_bins=num_bins)

    eval_rewards.append(total_reward)

print(f"\n🎯 EVALUACE (bez explorace):")
print(f"  Epizod: {eval_episodes}")
print(f"  Průměrná odměna: {np.mean(eval_rewards):.2f}")
print(f"  Std. dev: {np.std(eval_rewards):.2f}")
print(f"  Min: {np.min(eval_rewards):.2f}")
print(f"  Max: {np.max(eval_rewards):.2f}")

## 7. Uložení modelu

In [ ]:
# Vytvoř složku pro modely
models_dir = Path("multipong/ai/models")
models_dir.mkdir(parents=True, exist_ok=True)

model_path = models_dir / "q_table_pong.pkl"

# Ulož Q-tabulku
with open(model_path, "wb") as f:
    pickle.dump(Q, f)

print(f"✅ Model uložen: {model_path}")
print(f"   Velikost: {model_path.stat().st_size / 1024:.1f} KB")

## 8. Načtení a test

In [ ]:
# Vyzkoušej načtení modelu
with open(model_path, "rb") as f:
    Q_loaded = pickle.load(f)

print(f"✅ Model načten zpět!")
print(f"   Stavů v Q-tabulce: {len(Q_loaded)}")
print(f"   Kontrola: Q-tabulka je identická: {Q == Q_loaded}")